# Query Data with AWS Data Wrangler

**AWS Data Wrangler** is an open-source Python library that extends the power of the Pandas library to AWS connecting DataFrames and AWS data related services (Amazon Redshift, AWS Glue, Amazon Athena, Amazon EMR, Amazon QuickSight, etc).

* https://github.com/awslabs/aws-data-wrangler
* https://aws-data-wrangler.readthedocs.io

Built on top of other open-source projects like Pandas, Apache Arrow, Boto3, s3fs, SQLAlchemy, Psycopg2 and PyMySQL, it offers abstracted functions to execute usual ETL tasks like load/unload data from Data Lakes, Data Warehouses and Databases.

_Note that AWS Data Wrangler is simply a Python library that uses existing AWS Services.  AWS Data Wrangler is not a separate AWS Service.  You install AWS Data Wrangler through `pip install` as we will see next._

In [1]:
!pip install -q awswrangler==1.7.0

In [2]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

In [3]:
import awswrangler as wr

# Query Parquet from S3 with Push-Down Filters

Read Apache Parquet file(s) from from a received S3 prefix or list of S3 objects paths.

The concept of Dataset goes beyond the simple idea of files and enable more complex features like partitioning and catalog integration (AWS Glue Catalog): 

dataset (bool) – If True read a parquet dataset instead of simple file(s) loading all the related partitions as columns.

In [4]:
filter = lambda x: True if x["product_category"] == "Digital_Software" else False

In [5]:
path = 's3://{}/amazon-reviews-pds/parquet/'.format(bucket)
df_parquet_results = wr.s3.read_parquet(path,
                                     columns=['star_rating', 'product_category', 'review_body'],
                                     #filter=[("product_category", "=", "Digital_Software")],
                                     partition_filter=filter,
                                     dataset=True)
df_parquet_results.shape

(102084, 3)

In [6]:
df_parquet_results.head(5)

,star_rating,review_body,product_category
0,4,I successfully utilized the Quicken file conve...,Digital_Software
1,1,This program is a steaming pile of horse manur...,Digital_Software
2,1,"false sense of security, wouldn't recommend",Digital_Software
3,5,I purchased this about 3 weeks ago. I love it...,Digital_Software
4,5,I have been using Cook'n Recipe organizer soft...,Digital_Software


# Query Parquet from S3 in Chunks

Batching (chunked argument) (Memory Friendly):

Will enable the function to return a Iterable of DataFrames instead of a regular DataFrame.

There are two batching strategies on Wrangler:
* If chunked=True, a new DataFrame will be returned for each file in your path/dataset.
* If chunked=INTEGER, Wrangler will iterate on the data by number of rows equal to the received INTEGER.

P.S. chunked=True if faster and uses less memory while chunked=INTEGER is more precise in number of rows for each Dataframe.

In [7]:
path = 's3://{}/amazon-reviews-pds/parquet/'.format(bucket)
chunk_iter = wr.s3.read_parquet(path,
                                columns=['star_rating', 'product_category', 'review_body'],
                                # filters=[("product_category", "=", "Digital_Software")],
                                partition_filter=filter,
                                dataset=True,
                                chunked=True)

In [8]:
print(next(chunk_iter))

       star_rating                                        review_body  \
0                4  I successfully utilized the Quicken file conve...   
1                1  This program is a steaming pile of horse manur...   
2                1        false sense of security, wouldn't recommend   
3                5  I purchased this about 3  weeks ago. I love it...   
4                5  I have been using Cook'n Recipe organizer soft...   
...            ...                                                ...   
47979            4  File  electron federal return is free. But you...   
47980            3  The download went fine but the install hung on...   
47981            3  I've read some complaints about people who dow...   
47982            4  Very good product. I have been using Tax Cut f...   
47983            2  Okay first off the software works pretty good,...   

       product_category  
0      Digital_Software  
1      Digital_Software  
2      Digital_Software  
3      Digital_Soft

# Query the Glue Catalog (ie. Hive Metastore)
Get an iterator of tables.

In [9]:
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'
table_name_parquet = 'amazon_reviews_parquet'

In [10]:
for table in wr.catalog.get_tables(database="dsoaws"):
    print(table['Name'])

amazon_reviews_parquet
amazon_reviews_tsv


# Query from Athena
Execute any SQL query on AWS Athena and return the results as a Pandas DataFrame.  


In [11]:
%%time
df = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_parquet),
    database=database_name
)

CPU times: user 470 ms, sys: 41.4 ms, total: 512 ms
Wall time: 8.93 s


In [12]:
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,year,review_date,product_category
0,US,42562419,R1EAF5GW4MHN1N,B00NMPX58E,893965261,Photoshop Elements & Premiere Elements 13 - St...,5,5,7,N,Y,Awesome products,I have heard so much about this photoshop prod...,2014,2014-10-11,Digital_Software
1,US,43544559,R2C8JI7SE23T6H,B00JPRUDFQ,23010115,WordPerfect Office X7 Home and Student,5,0,0,N,Y,Five Stars,Wp is a great program<br /><br />Word perfect,2014,2014-10-11,Digital_Software
2,US,41551520,R171OZ99U5N2NV,B00M9GTJLY,103182180,Intuit Quicken Rental Property Manager 2015,1,61,62,N,N,Another disappointment,I have used Quicken of years and it seems to b...,2014,2014-10-11,Digital_Software
3,US,1744683,R39V4Z628T6GKK,B00E7XATMK,659550012,Learning QuickBooks 2014,1,0,0,N,Y,Not satisfied - first order was returned becau...,Not satisfied - first order was returned becau...,2014,2014-10-11,Digital_Software
4,US,30884389,R2558V53PHXK58,B008SCNLEY,643150354,Norton 360 2013 - 1 User / 3 PC,5,0,0,N,Y,Five Stars,Great product,2014,2014-10-11,Digital_Software


# Query from Athena in Chunks
Retrieving in chunks can help reduce memory requirements.

In [13]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_parquet),
    database='{}'.format(database_name),
    chunksize=64_000  # 64 KB Chunks
)

CPU times: user 322 ms, sys: 13.1 ms, total: 335 ms
Wall time: 6.93 s


In [14]:
print(next(chunk_iter))

     marketplace customer_id       review_id  product_id product_parent  \
0             US    41754720  R19OFJV91M7D8X  B000YMR61A      141393130   
1             US    51669529  R1I6G894K5AGG5  B000YMR61A      141393130   
2             US    24731012  R17OE43FFEP81I  B000YMR5X4      234295632   
3             US    16049580  R15MGDDK63B52Z  B000YMR61A      141393130   
4             US    46098046  R1GGJJA2R68033  B000YMNI2Q      847631772   
...          ...         ...             ...         ...            ...   
4995          US    17089950  R33TENVT3MZAHY  B003ZK5214      690806968   
4996          US    24596327  R3MSYUY4KJHVK5  B003SX16CM      190160525   
4997          US    36814854  R3LI4NFCKL5BP8  B004E9SKFA      494851576   
4998          US    15363863   RO9V9I92PWJI4  B004GB1YO0       47953870   
4999          US    10954841   RDTITG2AFTSAF  B004X0DDOI      958139200   

                                          product_title  star_rating  \
0                  TurboTax

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();